## Perps V3 Tutorial

This tutorial will walk you through the basics of using the Synthetix python SDK to interact with Perps V3. It will cover the following:
* Connecting to the Synthetix contracts
* Create a perps account
* Check sUSD balances (V3 and legacy)
* Fetch information about perps markets
* Deposit and withdraw collateral from perps accounts
* Commit an order
* Check the status of an order
* Settle an order using the Pyth price feed
 
Before you start, make sure you copy the `.env.example` file, rename to `.env` and fill in the required values. You can get an Infura API key [here](https://infura.io/).

In [ ]:
import os
import time
from synthetix import Synthetix
from dotenv import load_dotenv

load_dotenv()

In [ ]:
snx = Synthetix(
    provider_rpc=os.getenv('TESTNET_RPC'), # A Base Goerli RPC endpoint
    address=os.getenv('ADDRESS'), # A wallet address
    private_key=os.getenv('PRIVATE_KEY'), # A wallet private key
    network_id=84531, # Base Goerli
)

In [ ]:
# print the available account IDs
snx.perps.account_ids

In [ ]:
# if you don't have any account ID, create one
account_id = snx.perps.create_account(submit=True)

In [ ]:
# fetch the ids from the contract to confirm
snx.perps.get_account_ids()

In [ ]:
# fetch balances of sUSD and legacy sUSD
susd_balance = snx.get_susd_balance()

print(f"{susd_balance['balance']} sUSD")


In [ ]:
# display the markets
snx.perps.markets_by_name

In [ ]:
# fetch a fresh a market summary
market = snx.perps.get_market_summary(market_name='ETH')
print(market)

In [ ]:
# show your default account
print(f'Default account: {snx.perps.default_account_id}')

# get balances of collateral in your default account
balances = snx.perps.get_collateral_balances()
print(f'Collateral balances: {balances}')

# get info about your accounts margin
margin_info = snx.perps.get_margin_info()
print(f'Margin info: {margin_info}')

In [ ]:
# modify collateral
# the first argument is the amount of collateral to add or remove
modify_collateral = snx.perps.modify_collateral(500, market_id=0, submit=True)

In [ ]:
# get information about an open position
position = snx.perps.get_open_position(market_name='ETH')
print(position)

In [ ]:
# prepare an order commitment
# this prepares a 0.01 ETH order at the market price
order = snx.perps.commit_order(0.01, market_name='ETH')
print(order)

In [ ]:
# get your order
# after the transaction is confirmed, you will need to wait until `settlement_time` to settle the order
order = snx.perps.get_order()
print(order)
print(f"{order['settlement_time'] - time.time()} until settlement")

In [ ]:
# settle order
# this function will wait until the order is past the settlement time
# then, it will fetch the required price from pyth and settle the order
settle = snx.perps.settle_order(submit=True)
